In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/NLP/nlp-coursework-1')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
!pip install -q accelerate -U
!pip install -q simpletransformers
!pip install -q nlpaug

In [1]:
cd ..

/home/andyfu/Desktop/nlp-coursework-1


/home/andyfu/miniconda3/envs/nlp/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import wandb
import pandas as pd
import logging
import torch
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
from dont_patronize_me import DontPatronizeMe
from torch.utils.data import Dataset
import pandas as pd
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
import data_analysis_and_preprocessing.data_preprocessing as data_preprocessing
import nlpaug.augmenter.word as naw
# import gensim
# import transformers
# import nlpaug
# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action
# from nlpaug.util.file.download import DownloadUtil

# Disable wandb authorization request
os.environ["WANDB_START_METHOD"]="thread"
wandb.init(mode="disabled")

2024-03-04 13:49:29.790936: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 13:49:29.965238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 13:49:29.965324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 13:49:29.993316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 13:49:30.061458: I tensorflow/core/platform/cpu_feature_guar

In [25]:
random_seed = 42
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

device = 'cuda' if cuda_available else 'cpu'

print('Cuda available?', cuda_available)

PREPROCESSING_MODE = 'BASIC' # Choose between BASIC, MEDIUM and HEAVY
LOADING_MODE = 'k' # mode can be 'c', 'k', 'ck', ''

Cuda available? True


In [27]:
def preprocess_data(data):
    # data = data_preprocessing.remove_h_tags(data)
    data = data_preprocessing.remove_ampersands(data)
    data = data_preprocessing.remove_mentions(data)
    if PREPROCESSING_MODE != 'BASIC':
        data = data_preprocessing.lowercase(data)
    # data = data_preprocessing.remove_contractions(data)
    if PREPROCESSING_MODE == 'HEAVY':
        data = data_preprocessing.remove_multiple_quotations(data)
    # data = data_preprocessing.remove_extra_spaces(data)
    return data



def get_rows(data):
    rows = [] # will contain par_id, label and text
    for idx in data.index:
        parid = data.loc[idx]['par_id']
        instance = data.loc[idx]
        keyword = instance['keyword']
        country = instance['country']
        text = instance['text']
        if LOADING_MODE == 'c':
            text = country + ' | ' + text
        elif LOADING_MODE == 'k':
            text = keyword + ' | ' + text
        elif LOADING_MODE == 'ck':
            text = country + ' | ' + keyword + ' | ' + text
        print(type(parid), type(text), type(instance['label']))
        rows.append({
            'par_id':parid,
            'text':text,
            'label':instance['label']
        })
    return rows



def load_data(train_size=0.8, random_state=random_seed, rebuild_augmentation = False):
    dpm = DontPatronizeMe('.', '.')
    dpm.load_task1()
    trids = pd.read_csv('data/train_semeval_parids-labels.csv')
    teids = pd.read_csv('data/dev_semeval_parids-labels.csv')
    trids.par_id = trids.par_id.astype(str)
    teids.par_id = teids.par_id.astype(str)
    data = dpm.train_task1_df

    rows_train_val = [] # will contain par_id, label and text
    for idx in range(len(trids)):
        parid = trids.par_id[idx]
        # select row from original dataset to retrieve `text` and binary label
        instance = data.loc[data.par_id == parid]
        keyword = instance.keyword.values[0]
        text = instance.text.values[0]
        label = instance.label.values[0]
        country = instance.country.values[0]    
        rows_train_val.append({
            'par_id':parid,
            'keyword':keyword,
            'country':country,
            'text':text,
            'label':label
        })

    rows_train, rows_val = train_test_split(rows_train_val, train_size=train_size, random_state=random_state)

    rows_test = [] # will contain par_id, label and text
    for idx in range(len(teids)):
        parid = teids.par_id[idx]
        instance = data.loc[data.par_id == parid]
        keyword = instance.keyword.values[0]
        text = instance.text.values[0]
        label = instance.label.values[0]
        country = instance.country.values[0]
        rows_test.append({
            'par_id':parid,
            'keyword':keyword,
            'country':country,
            'text':text,
            'label':label
        })
    print(len(rows_train), len(rows_val), len(rows_test))

    train_set = pd.DataFrame(rows_train)
    val_set = pd.DataFrame(rows_val)
    test_set = pd.DataFrame(rows_test)

    caug = naw.ContextualWordEmbsAug(
            # option to choose from is "word2vec", "glove" or "fasttext"
            model_path='distilroberta-base',

            # options available are insert or substitute
            action='substitute')
    train_set_positive = train_set[train_set.label == 1]

    for idx, row in train_set_positive.iterrows():
        train_set_positive.at[idx, 'text'] = caug.augment(row.text)[0]

    train_set = pd.concat([train_set, train_set_positive])
    # reindex the train_set
    train_set = train_set.reset_index(drop=True)
    
    # train_set = preprocess_data(train_set)
    # val_set = preprocess_data(val_set)
    # test_set = preprocess_data(test_set)


    
    rows_train = get_rows(train_set)
    rows_val = get_rows(val_set)
    rows_test = get_rows(test_set)

    print(len(rows_train), len(rows_val), len(rows_test))

    train_dataset_raw = pd.DataFrame(rows_train)
    train_dataset_raw.to_csv('data/augmented_train.csv', index=False)
    pd.DataFrame(rows_val).to_csv('data/augmented_eval.csv', index=False)

    return pd.DataFrame(rows_train), pd.DataFrame(rows_val), pd.DataFrame(rows_test)

class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        train_input_ids = self.encodings.input_ids[idx]
        train_token_type_ids = self.encodings.token_type_ids[idx]
        train_attention_mask = self.encodings.attention_mask[idx]
        train_labels = self.encodings.label[idx]
        return {
            'input_ids': train_input_ids,
            'token_type_ids': train_token_type_ids,
            'attention_mask': train_attention_mask,
            'labels': train_labels
        }

    def __len__(self):
        return len(self.encodings.input_ids)

In [28]:
train_dataset_raw, eval_dataset_raw, test_dataset_raw = load_data(train_size=0.8, rebuild_augmentation = True)

6700 1675 2094
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 'numpy.int64'>
<class 'str'> <class 'str'> <class 

In [29]:
train_dataset_raw

,par_id,text,label
0,5423,disabled | Critics have even taken to dobbing ...,0
1,6890,in-need | Alexis and her family decided to don...,0
2,1394,"vulnerable | Mr Porter , do you think you will...",1
3,4445,"vulnerable | """""" This only serves to highlight...",0
4,7991,"hopeless | """""" I was nervous , but life has ta...",0
...,...,...,...
7322,5458,poor-families | President Uhuru Kenyatta and r...,1
7323,8198,"poor-families | ""Interacting with media, Adity...",1
7324,508,"in-need | """""" National Lutheran Church respect...",1
7325,6364,"refugee | ""An organisation representing Kashmi...",1


In [30]:
# Load the DeBERTa tokenizer
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

train_text = train_dataset_raw.text.values
eval_text = eval_dataset_raw.text.values
test_text = test_dataset_raw.text.values

max_length = 192

encoding_train = tokenizer(train_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_eval = tokenizer(eval_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)
encoding_test = tokenizer(test_text.tolist(), return_tensors='pt', padding="max_length", truncation=True, max_length=max_length)

encoding_train['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in train_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_eval['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in eval_dataset_raw['label'].tolist()], dtype=torch.float32)
encoding_test['label'] = torch.tensor([[0,1] if x == 1 else [1,0] for x in test_dataset_raw['label'].tolist()], dtype=torch.float32)

In [31]:
# Create an instance of the CustomDataset class
train_dataset = CustomDataset(encoding_train)
eval_dataset = CustomDataset(encoding_eval)
test_dataset = CustomDataset(encoding_test)

In [32]:
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)
model = model.to(device)

epochs = 8
lr = 1e-5
batch_size = 8

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    labels = np.argmax(labels, axis=-1)
    f1 = f1_score(labels, predictions, average='binary')
    return {'f1': f1}

trainingargs = TrainingArguments(
    learning_rate=lr,
    weight_decay=1e-2,
    output_dir='training_results',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    report_to=None,
    metric_for_best_model="f1",
    save_strategy='epoch',
    load_best_model_at_end=True,
    seed=random_seed,
    optim='adamw_torch',
)

trainer = Trainer(
    model=model,
    args=trainingargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# THESE CELLS HAVE BEEN RUN USING PREPROCESSING_MODE = 'MEDIUM'
assert PREPROCESSING_MODE == 'BASIC'

trainer.train()
# Here the best model has been already loaded
trainer.predict(test_dataset)

  0%|          | 0/7328 [00:00<?, ?it/s]

{'loss': 0.318, 'grad_norm': 0.4118777811527252, 'learning_rate': 9.317685589519652e-06, 'epoch': 0.55}


  0%|          | 0/210 [00:00<?, ?it/s]

{'eval_loss': 0.2852627635002136, 'eval_f1': 0.3937007874015748, 'eval_runtime': 12.3985, 'eval_samples_per_second': 135.097, 'eval_steps_per_second': 16.938, 'epoch': 1.0}
{'loss': 0.2597, 'grad_norm': 0.269204705953598, 'learning_rate': 8.635371179039303e-06, 'epoch': 1.09}
{'loss': 0.2197, 'grad_norm': 0.11000389605760574, 'learning_rate': 7.953056768558953e-06, 'epoch': 1.64}


  0%|          | 0/210 [00:00<?, ?it/s]

...

{'eval_loss': 0.658520519733429, 'eval_f1': 0.547945205479452, 'eval_runtime': 12.3754, 'eval_samples_per_second': 135.349, 'eval_steps_per_second': 16.969, 'epoch': 7.0}
{'loss': 0.0052, 'grad_norm': 0.002834279788658023, 'learning_rate': 1.1299126637554586e-06, 'epoch': 7.1}
{'loss': 0.0037, 'grad_norm': 0.003384869545698166, 'learning_rate': 4.4759825327510923e-07, 'epoch': 7.64}


  0%|          | 0/210 [00:00<?, ?it/s]

{'eval_loss': 0.6815210580825806, 'eval_f1': 0.5517241379310345, 'eval_runtime': 12.375, 'eval_samples_per_second': 135.353, 'eval_steps_per_second': 16.97, 'epoch': 8.0}
{'train_runtime': 1682.1507, 'train_samples_per_second': 34.846, 'train_steps_per_second': 4.356, 'train_loss': 0.09518153898775839, 'epoch': 8.0}


  0%|          | 0/262 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 5.213852 , -5.2630806],
       [-5.2332664,  5.0506744],
       [ 5.0967026, -5.3933954],
       ...,
       [ 5.2506857, -5.4356856],
       [ 5.314878 , -5.403362 ],
       [ 5.375198 , -5.571484 ]], dtype=float32), label_ids=array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32), metrics={'test_loss': 0.6111680269241333, 'test_f1': 0.5754189944134078, 'test_runtime': 15.4133, 'test_samples_per_second': 135.856, 'test_steps_per_second': 16.998})